In [1]:
!rm db/data_science.sqlite

rm: db/data_science.sqlite: No such file or directory


In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("db/alldata.csv", encoding = 'utf8')
# Cleans up the non-printable characters from description and company names
df.description.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
df.company.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
df.head()

,position,company,description,reviews,location
0,Development Director,ALS TDI,Development Director\nALS Therapy Development ...,NaN,"Atlanta, GA 30301"
1,An Ostentatiously-Excitable Principal Research...,The Hexagon Lavish,"Job Description\n\n""The road that leads to acc...",NaN,"Atlanta, GA"
2,Data Scientist,Xpert Staffing,"Growing company located in the Atlanta, GA are...",NaN,"Atlanta, GA"
3,Data Analyst,Operation HOPE,DEPARTMENT: Program OperationsPOSITION LOCATIO...,44.0,"Atlanta, GA 30303"
4,Assistant Professor -TT - Signal Processing & ...,Emory University,DESCRIPTION\nThe Emory University Department o...,550.0,"Atlanta, GA"


In [3]:
tags_list = ['<p>' ,'</p>' , '<p*>',
             '<ul>','</ul>',
             '<li>','</li>',
             '<br>',
             '<strong>','</strong>',
             '<span*>','</span>',
             '<a href*>','</a>',
             '<em>','</em>', '\n']

for tag in tags_list:
  df.replace(to_replace=tag, value=' ', regex=True, inplace=True)
# print(df)
df.columns = ['Position', 'Company', 'Description', 'Reviews', 'Location']
# Place NaN for the empty strings
df['Position'].replace('', np.nan, inplace=True)
# Drop the nulls
df.dropna(subset=['Position'], inplace=True)
# Save to all_clean.csv file
# df.to_csv("db/all_clean.csv")
df.head(2)

,Position,Company,Description,Reviews,Location
0,Development Director,ALS TDI,Development Director ALS Therapy Development I...,NaN,"Atlanta, GA 30301"
1,An Ostentatiously-Excitable Principal Research...,The Hexagon Lavish,"Job Description ""The road that leads to accom...",NaN,"Atlanta, GA"


In [4]:
df = df.drop(columns = ["Reviews"])
df = df.sort_values(by=["Location"], ascending=True)
df.head(2)

,Position,Company,Description,Location
6235,Data Engineer,Abbott Laboratories,"At Abbott, we're committed to helping people l...","Alameda, CA"
6013,Temporary Sr. Scientist I (Pre-Formulations),Exelixis Inc.,Responsible for all pre-formulation/formulatio...,"Alameda, CA"


In [5]:
# To remove all the digits from the location
df['City']=df['Location'].replace(regex=True,inplace=False,to_replace=r'\d',value=r'')

# To remove white space at the end of string:
df['City'] = df['City'].str.rstrip()
df.to_csv("db/all_cl.csv")
df.head(2)

,Position,Company,Description,Location,City
6235,Data Engineer,Abbott Laboratories,"At Abbott, we're committed to helping people l...","Alameda, CA","Alameda, CA"
6013,Temporary Sr. Scientist I (Pre-Formulations),Exelixis Inc.,Responsible for all pre-formulation/formulatio...,"Alameda, CA","Alameda, CA"


## 1 Cities of the Jobs

In [19]:
df_ci = df.groupby(['City']).size()
# dfnew = dfn.to_frame() put into new dataframe additonal .T would transpose it.
df_city = df_ci.to_frame()
df_city.columns = ["Count"]
df_city_s = df_city.sort_values(by=["Count"], ascending=False)
#To drop the index not to save as a new column df = df.reset_index(drop=True)
df_city= df_city_s.reset_index()
# To set the index as column df_city_s["City"] = df_city_s.index
df_city.to_csv("db/city.csv")
df_city.head(15)


,City,Count
0,"New York, NY",848
1,"Seattle, WA",777
2,"Cambridge, MA",694
3,"Boston, MA",629
4,"San Francisco, CA",564
5,"Chicago, IL",471
6,"San Diego, CA",412
7,"Washington, DC",340
8,"Mountain View, CA",277
9,"Atlanta, GA",269


## 2 Companies Hiring

In [20]:
df_co = df.groupby(['Company']).size()
# print(type(df_co))
df_company = df_co.to_frame()
df_company.columns = ["Count"]
df_company = df_company.sort_values(by=["Count"], ascending=False)

df_company= df_company.reset_index()
df_company.to_csv("db/company.csv")
df_company.head(15)

,Company,Count
0,Amazon.com,358
1,Ball Aerospace,187
2,Microsoft,137
3,Google,134
4,NYU Langone Health,77
5,Fred Hutchinson Cancer Research Center,70
6,KPMG,66
7,Lab126,50
8,Broad Institute,49
9,Facebook,49


## 3 Skill Set for the Jobs

In [21]:
skill_set = ['Python', 
             'JavaScript','Javascript', 
             'C++', 
             'Data Science', 
             'Mining', 
             'Machine Learning', 
             'MATLAB',
             'Tableau', 
             'Excel ', 
             'Java ',
             'SQL', 'sql',
             "SQLite",
             'Hadoop', 
             'AWS', 
             'Spark', 
             ' R ', ' R,', 
             'SAS', 
            ]

df_skills = pd.DataFrame(columns=skill_set)
for skill in skill_set:
    i=0
    for description in df['Description']:   
        if skill in description:
            i=i+1
#             print(i)
    df_skills.loc[0, skill] = i
df_skills = df_skills.T
df_skills.columns = ["Frequency"]
df_skills.loc['SQL']= df_skills.loc['SQL'] + df_skills.loc['sql']
df_skills.loc['JavaScript']= df_skills.loc['JavaScript'] + df_skills.loc['Javascript']
df_skills.loc[' R ']= df_skills.loc[' R '] + df_skills.loc[' R,']
df_skills = df_skills.sort_values(by=["Frequency"], ascending=False)
df_skills = df_skills.reset_index()
df_skills.columns = ["Skills", "Count"]
df_skills= df_skills[df_skills.Skills != " R,"]
df_skills= df_skills[df_skills.Skills != "sql"]
df_skills= df_skills[df_skills.Skills != "Javascript"]
df_skills.to_csv("db/skill.csv")
df_skills

,Skills,Count
0,Python,2759
1,SQL,1928
2,R,1616
4,C++,966
5,Spark,954
6,Machine Learning,931
7,Hadoop,918
8,Data Science,725
9,SAS,680
10,AWS,646


## 4 Education Requirements

In [22]:
educations = ['Master', 
             'PhD', 
             'BS', 
             'BA', 
             'MBA',
             'MS ',
             'MA '
            ]

df_education = pd.DataFrame(columns=educations)
for education in educations:
    i=0
    for description in df['Description']:   
        if education in description:
            i=i+1
#             print(i)
    df_education.loc[0, education] = i
df_education = df_education.T
df_education.columns = ["Frequency"]
df_education = df_education.sort_values(by=["Frequency"], ascending=False)
df_education = df_education.reset_index()
df_education.columns = ["Education", "Count"]
df_education.to_csv("db/education.csv")
df_education

,Education,Count
0,Master,1822
1,PhD,1682
2,MS,1302
3,BS,1244
4,BA,770
5,MA,257
6,MBA,206


## 5 Experience

In [23]:
experiences = [
    '6 years of experience ',
    '5 years of experience ',
    '4 years of experience ',
    '3 years of experience ',
    '2 years of experience ',
    '1 year of experience ',
    '5+ years of experience ', 
    '4+ years of experience ',
    '3+ years of experience ',
    '2+ years of experience ',
    '1+ years of experience ',
    'no experience'
             
            ]

df_exp = pd.DataFrame(columns=experiences)
for exp in experiences:
    i=0
    for description in df['Description']:   
        if exp in description:
            i=i+1
#             print(i)
    df_exp.loc[0, exp] = i
df_exp = df_exp.T
df_exp.columns = ["Frequency"]
df_exp = df_exp.sort_values(by=["Frequency"], ascending=False)
df_exp = df_exp.reset_index()
df_exp.columns = ["Experience", "Count"]
df_exp.to_csv("db/experience.csv")
# df_skills

df_exp

,Experience,Count
0,5+ years of experience,189
1,5 years of experience,175
2,3+ years of experience,148
3,2+ years of experience,143
4,3 years of experience,116
5,2 years of experience,104
6,4 years of experience,62
7,4+ years of experience,44
8,6 years of experience,28
9,1 year of experience,27


In [11]:
# Import SQLAlchemy `automap` and other dependencies
from sqlalchemy import create_engine, inspect

In [12]:
# Create the connection engine
engine = create_engine("sqlite:///db/data_science.sqlite")
conn = engine.connect()
from sqlalchemy.ext.declarative import declarative_base

## Data Scientists All Data Frame to SQLite

In [13]:
# from sqlalchemy.ext.declarative import declarative_base
# Base_Scientists = declarative_base()

# from sqlalchemy import * #Column, Integer, String, Float, Blob
# class Datumist(Base_Scientists):
#     __tablename__ = "Scientist"
#     id = Column(Integer, primary_key=True)
#     position = Column("Position",String(1255))
#     company = Column("Company",String(1255))
#     description = Column("Description", BLOB)
#     # numreview = Column("Numreview", String(1255))
#     location = Column("Location", String(255))
#     City = Column("City", String(255))
# Base_Scientists.metadata.drop_all(conn)
# Base_Scientists.metadata.create_all(conn)
# df.to_sql(Datumist.__tablename__, conn, index=False, if_exists="append")

## Cities to SQLite

In [14]:
Base_Cities = declarative_base()
from sqlalchemy import * #Column, Integer, String, Float, Blob
class Cities(Base_Cities):
    __tablename__ = "City"
    id = Column(Integer, primary_key=True)
    City = Column("City",String(255))
    Count = Column("Count",String(255))
Base_Cities.metadata.drop_all(conn)
Base_Cities.metadata.create_all(conn)  
df_city.to_sql(Cities.__tablename__, conn, index=False, if_exists="append")  

## Companies to SQLite

In [15]:
Base_Companies = declarative_base()
from sqlalchemy import * #Column, Integer, String, Float, Blob
class Companies(Base_Companies):
    __tablename__ = "Company"
    id = Column(Integer, primary_key=True)
    Company = Column("Company",String(255))
    Count = Column("Count",String(255))
Base_Companies.metadata.drop_all(conn)
Base_Companies.metadata.create_all(conn)  
df_company.to_sql(Companies.__tablename__, conn, index=False, if_exists="append")  

## Skills to SQLite

In [16]:
Base_Skills = declarative_base()
from sqlalchemy import * #Column, Integer, String, Float, Blob
class Skills(Base_Skills):
    __tablename__ = "Skill"
    id = Column(Integer, primary_key=True)
    Skill = Column("Skills",String(255))
    Count = Column("Count",String(255))
Base_Skills.metadata.drop_all(conn)
Base_Skills.metadata.create_all(conn)  
df_skills.to_sql(Skills.__tablename__, conn, index=False, if_exists="append")  

## Education to SQLite

In [17]:
Base_Education = declarative_base()
from sqlalchemy import * #Column, Integer, String, Float, Blob
class Education(Base_Education):
    __tablename__ = "Education"
    id = Column(Integer, primary_key=True)
    Education = Column("Education",String(255))
    Count = Column("Count",String(255))
Base_Education.metadata.drop_all(conn)
Base_Education.metadata.create_all(conn)  
df_education.to_sql(Education.__tablename__, conn, index=False, if_exists="append") 

## Years of Experience to SQLite

In [18]:
Base_Experiences = declarative_base()
from sqlalchemy import * #Column, Integer, String, Float, Blob
class Experiences(Base_Experiences):
    __tablename__ = "Experience"
    id = Column(Integer, primary_key=True)
    Skill = Column("Experience",String(255))
    Count = Column("Count",String(255))
Base_Experiences.metadata.drop_all(conn)
Base_Experiences.metadata.create_all(conn)  
df_exp.to_sql(Experiences.__tablename__, conn, index=False, if_exists="append")  